Previously deployed this on ShinyApp, now trying it with Python Dash. Deploy on localhost after downloading.

Features to include:
1. User input bus servive
2. User input Bus stop ID
3. Bus Arrival Tab (Table showing specific bus arrival at queried bus stop)
4. Bus Arrival Table (Table showing all bus arrivals at queried bus stop)
5. Bus Route Tab (Map showing bus route)

In [79]:
import json
import urllib
import httplib2 as http 
from urllib.parse import urlparse
import os
import pandas as pd

from datetime import datetime

In [2]:
# Read in Token
with open("../python_repo_doc/bus_arrival_doc/LTA_AccountKey") as k:
    key = k.readlines()

In [ ]:
# LTA Datamall API
headers = {"AccountKey": key[0],
           "accept": "application/json"}

uri = "http://datamall2.mytransport.sg/"

# Change path there to change retrieval
method = "GET"
body = ""
h = http.Http()

In [217]:
# Define function to retrieve bus arrivals from LTA Datamall
def get_arrivals(bus_stop_code):
    
    path = "ltaodataservice/BusArrivalv2?BusStopCode=" + str(bus_stop_code)

    target = urlparse(uri + path)  
    
    reponse, content = h.request(
        target.geturl(),
        method,
        body,
        headers
    )

    jsonObj = json.loads(content)
    
    return jsonObj

In [219]:
# Get bus arrival timings from specific bus stop code.
jsonObj = get_arrivals(bus_stop_code = "05013")

# Filter out bus arrivals from JSON object.
service_num = [bus["ServiceNo"] for bus in jsonObj["Services"]]
service_num_1 = [bus["NextBus"]["EstimatedArrival"] for bus in jsonObj["Services"]]
service_num_2 = [bus["NextBus2"]["EstimatedArrival"] for bus in jsonObj["Services"]]
service_num_3 = [bus["NextBus3"]["EstimatedArrival"] for bus in jsonObj["Services"]]

bus_list = pd.DataFrame({"Service Number": service_num,
                         "Next Bus": service_num_1,
                         "Next Bus 2": service_num_2,
                         "Next Bus 3": service_num_3})

# Define function to convert str time to datetime time.
def convert_time(t):
    
    try:
        dt_t = datetime.strptime(t[:-6], "%Y-%m-%dT%H:%M:%S")
        
        if dt_t > datetime.now():
            td = dt_t - datetime.now()
            
            if td.seconds < 60:
                return "Arr"
            else:
                return round(td.seconds/60)
        else:
            return ""
    
    except:
        return ""

bus_list["Next Bus"] = bus_list["Next Bus"].apply(convert_time)
bus_list["Next Bus 2"] = bus_list["Next Bus 2"].apply(convert_time)
bus_list["Next Bus 3"] = bus_list["Next Bus 3"].apply(convert_time)

# Print out data frame coming bus arrival timings in minutes
print("After Conversion:\n", bus_list.sort_values(by="Service Number"))

After Conversion:
    Service Number Next Bus  Next Bus 2 Next Bus 3
0              12        5          18           
1             12e       12          42           
2             143        5          16         26
3             197      Arr          11         26
4               2                   14         26
5              33        6          27         42
6              61        5          14         27
7              63        4          18         38
8              80        5          17         33
9             961        5          18         30
10            970        6          19         36
